In [1]:
# Import libraries.

import json

from IPython.display import Markdown, display

In [2]:
# Define parameters.

PROMPTS = "../data/prompts/zeroshot/taskvine_zeroshot_gpt4o_1.json"

In [3]:
# Analyze a singular prompt.

def analyze(prompt: dict):
    print(prompt["content"])
    display(Markdown(prompt["response"]))

In [4]:
# Load prompts from file.
with open(PROMPTS, "r") as fp:
    prompts = json.load(fp)

# Perform the zeroshot prompting.
for prompt in prompts:
    analyze(prompt)

Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date".


```python
from ndcctools.taskvine import Manager, Task

# Create a Manager
manager = Manager()

# Define the command to get the current date and redirect it to "current_date"
command = "date > current_date"

# Create a Task with the command
task = Task(command)

# Specify the output file
task.set_output_files(["current_date"])

# Add the task to the manager
manager.add_task(task)

# Wait for all tasks to complete
manager.wait_for_all()
```

Write a Python TaskVine application to count the number of times the keywords "wealth", "nation", "labour", "price", "nature", and "commodity" appear in Adam Smith's Wealth of Nations. The text is available at "https://www.gutenberg.org/cache/epub/3300/pg3300.txt".


```python
import urllib.request
from ndcctools.taskvine import Manager, Worker, Task

# Define the URL of the text
url = "https://www.gutenberg.org/cache/epub/3300/pg3300.txt"

# Keywords to search for
keywords = ["wealth", "nation", "labour", "price", "nature", "commodity"]

def download_text(url):
    # Download the text file
    response = urllib.request.urlopen(url)
    return response.read().decode('utf-8')

def split_text(text, lines_per_chunk):
    # Split the text into chunks
    lines = text.splitlines()
    for i in range(0, len(lines), lines_per_chunk):
        yield " ".join(lines[i:i+lines_per_chunk])

def count_keywords(chunk, keywords):
    # Count keywords in chunk
    counts = {keyword: chunk.lower().count(keyword) for keyword in keywords}
    return counts

text = download_text(url)
chunks = list(split_text(text, 1000))  # Splitting text into chunks of 1000 lines

# Define the TaskVine Task Manager
with Manager() as manager:
    # Add a task for each chunk
    for i, chunk in enumerate(chunks):
        task = Task(command=f"python3 -c 'import pickle; "
                            f"c=pickle.load(open(\"chunk_{i}.pkl\", \"rb\")); "
                            f"counts={{k:c.lower().count(k) for k in {keywords}}}; "
                            f"pickle.dump(counts, open(\"result_{i}.pkl\", \"wb\"))'")
        
        # Serialize chunk to a file
        with open(f"chunk_{i}.pkl", "wb") as f:
            pickle.dump(chunk, f)
        
        task.upload_files([f"chunk_{i}.pkl"])
        task.download_output_files([f"result_{i}.pkl"])
        manager.submit(task)
    
    # Wait for all tasks to complete
    manager.wait_for_all()

    # Collect results
    total_counts = {keyword: 0 for keyword in keywords}

    for i in range(len(chunks)):
        with open(f"result_{i}.pkl", "rb") as f:
            result = pickle.load(f)
            for keyword, count in result.items():
                total_counts[keyword] += count

    # Print the total counts
    print("Keyword counts in 'The Wealth of Nations':")
    for keyword, count in total_counts.items():
        print(f"{keyword}: {count}")
```


Write a Python TaskVine application to run my program "script.sh" with various input arguments and organizes all output data. "script.sh" takes, as input, an integer and two files. "script.sh" outputs a file named "output{i}.txt" where "{i}" is the input integer. The application must run "script.sh" with input integers 1-1000 and input files "file1.input" and "file2.input" which are stored in the directory "script_inputs". The application must store all output files in a directory named "script_outputs".


...

```python
import numpy as np
from ndcctools import Taskvine

# Define TaskVine library
matrix_library = Taskvine.Library('matrix_library')

# Define a function to invert a matrix
@matrix_library.function(name='invert_matrix', version=1)
def invert_matrix(matrix):
    return np.linalg.inv(matrix)

# Define a function to transpose a matrix
@matrix_library.function(name='transpose_matrix', version=1)
def transpose_matrix(matrix):
    return np.transpose(matrix)

# Function to generate random matrices and perform inversion and transposition
def process_matrices(m, n, num_matrices):
    for _ in range(num_matrices):
        matrix = np.random.rand(m, n)
        print("Original Matrix:")
        print(matrix)

        # Perform inversion
        task = Taskvine.Task()
        inv_result = task.call(invert_matrix, matrix)
        inverse_matrix = task.get_result(inv_result)
        print("Inverted Matrix:")
        print(inverse_matrix)

        # Perform transposition
        task = Taskvine.Task()
        trans_result = task.call(transpose_matrix, matrix)
        transposed_matrix = task.get_result(trans_result)
        print("Transposed Matrix:")
        print(transposed_matrix)

# Main code to run the application
if __name__ == "__main__":
    m = int(input("Enter number of rows (m): "))
    n = int(input("Enter number of columns (n): "))
    i = int(input("Enter number of matrices (i): "))
    
    # Initialize Taskvine
    Taskvine.Server.initialize()
    
    try:
        process_matrices(m, n, i)
    finally:
        Taskvine.Server.shutdown()
```


Write a Python Taskvine application to fetch the titles of multiple news webpages. The webpage contents must be downloaded asynchronously using the TaskVine futures executor. Print the URL and title of each webpage in order of the completion of each task. The webpages I am interested in are "https://abcnews.com", "https://apnews.com", "https://axios.com", "https://cnn.com", "https://nbcnews.com", "https://nytimes.com", "https://reuters.com", "https://washingtonpost.com", "https://wsj.com".


```python
import requests
from bs4 import BeautifulSoup
from ndcctools.taskvine import Executor

# List of URLs to fetch
urls = [
    "https://abcnews.com",
    "https://apnews.com",
    "https://axios.com",
    "https://cnn.com",
    "https://nbcnews.com",
    "https://nytimes.com",
    "https://reuters.com",
    "https://washingtonpost.com",
    "https://wsj.com"
]

def fetch_title(url):
    """Fetch the HTML content of a webpage and extract its title."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.title.string if soup.title else 'No title found'
        return url, title
    except requests.RequestException as e:
        return url, f'Error: {e}'

def main():
    with Executor() as executor:
        futures = [executor.submit(fetch_title, url) for url in urls]
        for future in executor.as_completed(futures):
            url, title = future.result()
            print(f"URL: {url}, Title: {title}")

if __name__ == '__main__':
    main()
```


Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date". To prevent an unwanted worker from connecting to the manager, use the password stored in the file "taskvine.password". Further, encrypt communication between the manager and worker with SSL using the "key.pem" and "certificate.pem" files.


```python
import ndcctools.taskvine as vine

# Create a TaskVine manager
manager = vine.Manager()

# Set the manager password
manager.set_password_file("taskvine.password")

# Set SSL options for encryption
manager.set_ssl_options("certificate.pem", "key.pem")

# Define the command to get the date
command = "date > current_date"

# Create a TaskVine task
task = vine.Task(command)

# Add the task to the manager
manager.submit(task)

# Run the manager
manager.run()
```